# Feature Engineering

## Setup

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import pprint
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional

In [ ]:
# FOLDERS
OUTPUT_FOLDER = "../Data/Output"
INPUT_FOLDER = "../Data/Input"

INPUT_CB_FOLDER = os.path.join(INPUT_FOLDER, "CB_FINAL")
OUTPUT_CB_FOLDER = os.path.join(OUTPUT_FOLDER, "Crunchbase")
OUTPUT_ECON_FOLDER = os.path.join(OUTPUT_FOLDER, "Economic")
OUTPUT_NLP_FOLDER = os.path.join(OUTPUT_FOLDER, "NLP")
OUTPUT_FINAL_FOLDER = os.path.join(OUTPUT_FOLDER, "Final")

output_contents_cb = os.listdir(INPUT_CB_FOLDER)
output_contents_econ = os.listdir(OUTPUT_ECON_FOLDER)
output_contents_nlp = os.listdir(OUTPUT_NLP_FOLDER)

print(output_contents_cb)
print(output_contents_econ)
print(output_contents_nlp)

### Load Initial Datasets

#### CB

In [ ]:
early_df_path = os.path.join(INPUT_CB_FOLDER, "cb_early_features.csv")
rounds_df_path = os.path.join(INPUT_CB_FOLDER, "cb_post_funding_rounds.csv")
outcomes_df_path = os.path.join(INPUT_CB_FOLDER, "cb_outcomes.csv")

early_df = pd.read_csv(early_df_path)
rounds_df = pd.read_csv(rounds_df_path)
outcomes_df = pd.read_csv(outcomes_df_path)

In [ ]:
early_df.info()
rounds_df.info()
outcomes_df.info()

#### Econ

In [ ]:
econ_df_path = os.path.join(OUTPUT_ECON_FOLDER, "economic_data.csv")
econ_df = pd.read_csv(econ_df_path)

In [ ]:
econ_df.info()

#### NLP

In [ ]:
nlp_df_path = os.path.join(OUTPUT_NLP_FOLDER, "perplexity.csv")
nlp_df = pd.read_csv(nlp_df_path)

In [ ]:
nlp_df.info()

## Individual Sets

### Crunchbase

#### Constants

In [ ]:
# CONSTANTS
SUCCESS_YEARS_THRESHOLD = 5

# top unviversities based on most startup founders
TOP_SCHOOLS = [ 
    "Stanford University", "Stanford",
    "University of California, Berkeley", "UC Berkeley", "Berkeley",
    "Harvard University", "Harvard",
    "Massachusetts Institute of Technology", "MIT",
    "University of Pennsylvania", "UPenn", "Penn",
    "Cornell University", "Cornell",
    "University of Michigan, Ann Arbor", "University of Michigan", "Michigan", "UMich",
    "University of Texas at Austin", "UT Austin",
    "Yale University", "Yale",
    "University of California, Los Angeles", "UCLA", "Los Angeles",
    "Princeton University", "Princeton",
    "Columbia University", "Columbia",
    "University of Illinois, Urbana-Champaign", "UIUC", "Illinois",
    "University of Southern California", "USC", "Southern California",
    "University of Wisconsin-Madison", "UW Madison", "Wisconsin",
    "New York University", "NYU",
    "Brown University", "Brown",
    "Duke University", "Duke",
    "Carnegie Mellon University", "Carnegie Mellon", "CMU",
    "Northwestern University", "Northwestern"
]

CS_ENGINEERING_DEGREES = [
    "Computer Science", "CS", "Data Science", "Software", "Engineering", "Finance", "Business Administration"
]

# RELEVANT_POSTIONS = ['founder', 'ceo', 'chief executive officer', 'cto',
#                     'chief technology officer', 'coo', 'chief operating officer',
#                     'cfo', 'chief financial officer', 'cso', 'chief strategy officer']

RELEVANT_POSTIONS = ['founder']

STARTUP_HUB_CITIES = [
    "San Francisco", "New York", "Boston", "Los Angeles", "Austin"
]

TOP_VC = [
    "Sequoia Capital", "Sequoia",
    "Andreessen Horowitz", "a16z",
    "Accel", "Accel Partners",
    "Tiger Global Management", "Tiger Global",
    "Lightspeed Venture Partners", "Lightspeed",
    "Benchmark", "Benchmark Capital",
    "New Enterprise Associates", "NEA",
    "Bessemer Venture Partners", "Bessemer",
    "Kleiner Perkins", "KPCB",
    "Greylock Partners", "Greylock",
    "Index Ventures",
    "General Catalyst", "General Catalyst Partners",
    "Founders Fund",
    "First Round Capital", "First Round",
    "Y Combinator", "YC",
    "Peter Thiel",
    "Reid Hoffman",
    "Elon Musk",
    "Marc Andreessen",
    "Chris Sacca",
    "Paul Graham",
    "Ron Conway",
    "Naval Ravikant",
    "Max Levchin",
    "David Sacks",
    "Vinod Khosla",
    "Paul Buchheit",
    "Jessica Livingston",
    "Steve Anderson",
    "Aydin Senkut"
]

VALID_ACTIVE_STATUSES = ['operating', 'ipo', 'acquired']

EARLY_FUNDING_TYPES = {
    'series_a', 'series_b', 'series_c', 'series_d',
    'series_e', 'series_f', 'series_g', 'series_h',
    'series_i', 'series_j', 'series_unknown', 'corporate_round'
}

VALID_GENDER_LABELS = {
    'male', 'female', 'ftm', 'androgyne', 'transgender_woman',
    'non_binary', 'agender', 'transgender_person', 'other',
    'androgynous', 'genderqueer', 'gender_nonconforming'
}

bucket_to_categories = {
    "Life Sciences": [
        "Bioinformatics", "Biopharma", "Biotechnology", "Health Care",
        "Health Diagnostics", "Medical", "Medical Device", "Pharmaceutical",
        "Life Science", "Wearables"
    ],
    "Fintech": [
        "Finance", "Financial Services", "FinTech", "Health Insurance", "InsurTech"
    ],
    "Consumer Goods": [
        "Apparel", "Consumer Electronics", "E-Commerce", "Fashion", "Retail",
        "Retail Technology", "Food and Beverage"
    ],
    "Technology": [
        "Cloud Computing", "DevOps", "Enterprise Software", "Information Technology",
        "Legal", "Marketing", "Mobile", "Mobile Apps", "SaaS", "PaaS", "Sales", "Software", "Blockchain", "Cryptocurrency", "Web3",
        "Artificial Intelligence (AI)", "Data Management", "Machine Learning", "Natural Language Processing", 
        "Electronics", "Industrial Automation", "Internet of Things",  "Robotics", "Semiconductor", "Sensor", "Supply Chain Management",
        "Cyber Security", "Security", "Drones", "Hardware",
        "EdTech", "Nanotechnology"
    ],
    "Cleantech": [
        "Battery", "Clean Energy", "CleanTech", "Energy",
        "Environmental Consulting", "GreenTech", "Oil and Gas", "Solar"
    ],
    "Transportation": [
        "Aerospace", "Automotive", "Electric Vehicle",
        "Logistics", "Space Travel", "Transportation", "Travel"
    ],
    "Media Entertainment and Gaming": [
        "Augmented Reality", "Gaming", "Media and Entertainment", "Online Games",
        "Social Media", "Sports", "Video Games", "Video Streaming", "Virtual Reality"
    ],
    "Telecom": [
        "Telecommunications"
    ],
    "Real Estate": [
        "PropTech", "Real Estate"
    ]
}

#### Filtering

In [ ]:
num_unique_col = early_df['org_uuid'].nunique()
print(num_unique_col)

In [ ]:
"""
Double check that only funding rounds within 5 years of funding
"""
# Ensure date columns are parsed first
early_df['announced_on'] = pd.to_datetime(early_df['announced_on'], errors='coerce')
early_df['founded_on'] = pd.to_datetime(early_df['founded_on'], errors='coerce')

# Drop rows with NaN dates
early_df = early_df.dropna(subset=['founded_on'])

# Reapply 5-year cutoff as a redundancy to ensure early_df is clean
early_df = early_df[
    early_df['announced_on'].isna() |
    ((early_df['announced_on'] - early_df['founded_on']).dt.days <= 1825)
]

In [ ]:
def map_industries(category_str, mapping_dict):
    """
    Drops startups with no relevant industry match, create industry column
    """
    if pd.isna(category_str):
        return []
    categories = [x.strip() for x in category_str.split(',')]
    matched = []
    for key, terms in mapping_dict.items():
        if any(cat in terms for cat in categories):
            matched.append(key)
    return matched

In [ ]:
def is_founder_role(title):
    """
    Filter for executive job roles and drop everything else
    """
    if isinstance(title, str):
        title = title.lower()
        return any(k in title for k in RELEVANT_POSTIONS)
    return False


In [ ]:
def is_founder_of_org(description, org_name):
    if not isinstance(description, str) or not isinstance(org_name, str):
        return False

    desc = description.lower()
    org = re.escape(org_name.lower())  # escape for special chars like '+' or '&'

    # Tokenize description (split on non-word chars to handle punctuation)
    tokens = re.split(r'\W+', desc)
    
    # Iterate through positions where 'founder' appears
    founder_indices = [i for i, word in enumerate(tokens) if 'founder' in word]

    for i in founder_indices:
        # Define the window around "founder"
        window_start = max(i - 3, 0)
        window_end = i + 4  # inclusive of 5 after

        context_window = tokens[window_start:window_end]
        if any(org in token for token in context_window):
            return True
    return False

string = 'Kevin is the CEO and cofounder of Instagram a community of more than 300 million who capture and share the worlds moments on the service He is responsible for the companys overall vision and strategy as well as daytoday operations   Prior to founding Instagram Kevin was part of the startup Odeo which later became Twitter and spent two years at Google working on products such as Gmail and Google Reader He graduated from Stanford University with a bachelor of science in management science and engineering'
print(is_founder_of_org(string, "Instagram"))

In [ ]:
# founders_df = early_df[early_df['featured_job_title'].apply(is_founder_role)].copy()
# founders_from_desc = early_df[
#     early_df['person_description'].apply(has_founder_in_description)
# ].copy()
# founders_df = pd.concat([founders_df, founders_from_desc], ignore_index=True).drop_duplicates(subset=['org_uuid', 'person_uuid'])
# valid_orgs = founders_df['org_uuid'].unique()


# Title-based founder rows
founders_from_title = early_df[
    early_df['featured_job_title'].apply(is_founder_role)
]

# Description-based founder rows
founders_from_desc = early_df[
    early_df.apply(lambda row: is_founder_of_org(row['person_description'], row['org_name']), axis=1)
]

# Combine and deduplicate by person and org
founders_df = pd.concat([founders_from_title, founders_from_desc], ignore_index=True)
founders_df = founders_df.drop_duplicates(subset=['org_uuid', 'person_uuid'])

# Keep only founders with description, gender, and either school or degree
filtered_founders = founders_df[
    founders_df['person_description'].notna() &
    founders_df['person_gender'].notna() &
    (
        founders_df['institution_name'].notna() |
        founders_df['subject'].notna() |
        founders_df['degree_type'].notna()
    )
].copy()

# Now collect orgs with at least one true founder
valid_orgs = filtered_founders['org_uuid'].unique()


In [ ]:
# filter for companies with non-null organization descriptions
early_df = early_df[early_df['org_description'].notna() & (early_df['org_description'].str.strip() != '')].copy()

# map industries to 9 main categories
early_df['matched_industries'] = early_df['category_list'].apply(lambda x: map_industries(x, bucket_to_categories))
early_df = early_df[early_df['matched_industries'].map(len) > 0].copy()
early_df['industry'] = early_df['matched_industries'].apply(lambda x: ','.join(set(x)))

# anchored list
base_orgs = early_df[['org_uuid', 'org_name', 'city', 'founded_on', 'industry', 'org_description']].drop_duplicates('org_uuid')

early_df.info()

In [ ]:
num_unique_col = early_df['org_uuid'].nunique()
print(num_unique_col)

#### Feat Engineering

In [ ]:
"""
Helper Function
"""
EXCLUDE_GENDER_LABELS = {'unknown', 'not_provided', 'none', '', None, np.nan}

# Shannon entropy
# p = proportion of founders in gender category i
# H is higher when gender distribution is more uniform (diverse)
def compute_gender_diversity(genders):
    clean_genders = [
        str(g).strip().lower()
        for g in genders
        if str(g).strip().lower() not in EXCLUDE_GENDER_LABELS
    ]
    if not clean_genders:
        return None
    
    counts = pd.Series(clean_genders).value_counts(normalize=True)
    probs = counts.values
    entropy = -np.sum(probs * np.log(probs))
    return entropy / np.log(len(probs)) if len(probs) > 1 else 0.0 # normalized entropy (0–1), only if multiple distinct categories

In [ ]:
"""
Founder and Education Features
"""
# Aggregate founder-level features at org level
founder_feats = filtered_founders.groupby('org_uuid').agg(
    founder_count=('person_uuid', 'nunique'),
    founder_gender_diversity=('person_gender', compute_gender_diversity),
    known_gender_count=('person_gender', lambda x: sum(pd.notna(x))),
    has_top_school_founder=('institution_name', lambda x: any(
        str(inst).strip() in TOP_SCHOOLS for inst in x if pd.notna(inst)
    )),
    num_optimal_degrees=('subject', lambda x: sum([
        1 if any(cs.lower() in str(subj).strip().lower() for cs in CS_ENGINEERING_DEGREES) else 0
        for subj in x if pd.notna(subj)
    ]))
).reset_index()

# Normalize optimal degree count per founder
founder_feats['optimal_degree_ratio'] = founder_feats['num_optimal_degrees'] / founder_feats['founder_count']

# is_repeat_founder
repeat_counts = early_df.groupby('person_uuid')['org_uuid'].nunique().reset_index()
repeat_founders = repeat_counts[repeat_counts['org_uuid'] > 1]['person_uuid']
early_df['is_repeat_founder'] = early_df['person_uuid'].isin(repeat_founders)
repeat_founder_df = early_df.groupby('org_uuid')['is_repeat_founder'].max().reset_index()

# Concatenate all founder descriptions into one string per org
founder_desc_df = filtered_founders.groupby('org_uuid')['person_description'].apply(
    lambda x: ' '.join([str(s) for s in x if pd.notna(s)])
).reset_index().rename(columns={'person_description': 'founder_description_blob'})



In [ ]:
"""
Founding Timing Features
"""
early_df['founded_on'] = pd.to_datetime(early_df['founded_on'], errors='coerce')
early_df['announced_on'] = pd.to_datetime(early_df['announced_on'], errors='coerce')

# First funding round timing
funding_group = early_df.groupby('org_uuid')['announced_on'].min().reset_index()
funding_group = funding_group.rename(columns={'announced_on': 'first_funding_date'})

timing_df = early_df[['org_uuid', 'founded_on']].drop_duplicates().merge(funding_group, on='org_uuid', how='left')
timing_df['age_at_first_funding'] = (timing_df['first_funding_date'] - timing_df['founded_on']).dt.days / 30
timing_df['first_funding_delay'] = timing_df['age_at_first_funding']
timing_df['founded_year'] = timing_df['founded_on'].dt.year


In [ ]:
"""
Early Funding Dynamics
"""
# Ensure datetime formatting
early_df['announced_on'] = pd.to_datetime(early_df['announced_on'], errors='coerce')
early_df['founded_on'] = pd.to_datetime(early_df['founded_on'], errors='coerce')

# Base: one row per unique funding round
unique_early_rounds = early_df.drop_duplicates(subset='funding_round_uuid').copy()

# --- Months since founding for each early round ---
unique_early_rounds['months_since_founding'] = (
    (unique_early_rounds['announced_on'] - unique_early_rounds['founded_on']).dt.days / 30
)

# --- Calculate Early Series Count ---
print("Calculating early series count...")
early_series_count_df = unique_early_rounds.groupby('org_uuid').size().reset_index(name='early_series_count')

# --- Calculate Average Time To an Early Round (Corrected Name) ---
print("Calculating average time to early round...")
avg_time_to_round_df = unique_early_rounds.groupby('org_uuid')['months_since_founding'].mean().reset_index(name='avg_time_to_early_round_months')

# --- Calculate Average Time BETWEEN Early Rounds (Correct Logic) ---
print("Calculating average time between early rounds...")
unique_early_rounds = unique_early_rounds.sort_values(['org_uuid', 'announced_on'])
unique_early_rounds['time_diff_months'] = unique_early_rounds.groupby('org_uuid')['months_since_founding'].diff()
avg_time_between_df = unique_early_rounds.dropna(subset=['time_diff_months']) \
    .groupby('org_uuid')['time_diff_months'].mean().reset_index(name='avg_time_between_rounds')

# --- Has Any Early Rounds ---
valid_rounds = unique_early_rounds.dropna(subset=['announced_on'])[['org_uuid']]
has_funding_df = valid_rounds.groupby('org_uuid').size().reset_index(name='num_early_rounds')
has_funding_df['has_funding_data'] = True

# Ensure all orgs represented
all_orgs = early_df[['org_uuid']].drop_duplicates()
has_funding_df = all_orgs.merge(has_funding_df, on='org_uuid', how='left')
has_funding_df['num_early_rounds'] = has_funding_df['num_early_rounds'].fillna(0).astype(int)
has_funding_df['has_funding_data'] = has_funding_df['has_funding_data'].fillna(False)

# --- Disclosed Funding ---
valid_amounts = unique_early_rounds.dropna(subset=['raised_amount_usd'])[['org_uuid', 'raised_amount_usd']]
known_amounts_df = valid_amounts.groupby('org_uuid').size().reset_index(name='num_disclosed_rounds')
known_amounts_df['has_disclosed_funding'] = True

known_amounts_df = all_orgs.merge(known_amounts_df, on='org_uuid', how='left')
known_amounts_df['num_disclosed_rounds'] = known_amounts_df['num_disclosed_rounds'].fillna(0).astype(int)
known_amounts_df['has_disclosed_funding'] = known_amounts_df['has_disclosed_funding'].fillna(False)

# --- Known Investor Flag ---
print("Checking for known investors...")
def has_known_investor(investors_str, top_vcs):
    if pd.isna(investors_str):
        return False
    return any(vc.lower() in investors_str.lower() for vc in top_vcs)

investors_per_org = early_df.groupby('org_uuid')['investor_name'].apply(
    lambda x: ' '.join(str(s) for s in x if pd.notna(s))
).reset_index()

investors_per_org['has_known_investor'] = investors_per_org['investor_name'].apply(
    lambda x: has_known_investor(x, TOP_VC)
)

known_investor_df = investors_per_org[['org_uuid', 'has_known_investor']]

# --- Total Unique Investors ---
early_unique_inv = early_df[['org_uuid', 'funding_round_uuid', 'investor_count']].drop_duplicates()
investor_count_df = early_unique_inv.groupby('org_uuid')['investor_count'].sum(min_count=1).reset_index()

# --- Funding Velocity (within first 5 years) ---
print("Calculating funding velocity...")
def compute_funding_velocity(df, time_window_months=60):
    df = df.drop_duplicates(subset='funding_round_uuid').copy()
    df['months_since_founding'] = (df['announced_on'] - df['founded_on']).dt.days / 30
    within_window = df[df['months_since_founding'] <= time_window_months][['org_uuid', 'announced_on']].drop_duplicates()
    velocity_df = within_window.groupby('org_uuid').size().reset_index(name='early_rounds_5yr')
    velocity_df['funding_velocity'] = velocity_df['early_rounds_5yr'] / (time_window_months / 12)  # rounds/year
    return velocity_df[['org_uuid', 'funding_velocity']]

funding_velocity_df = compute_funding_velocity(early_df)

# --- First Funding Amount Buckets ---
print("Bucketizing first funding amounts...")
def bucketize_funding_amounts(df):
    df = df.drop_duplicates(subset='funding_round_uuid')
    df = df.sort_values('announced_on')
    first_funding = df.groupby('org_uuid').first().reset_index()
    buckets = pd.cut(
        first_funding['raised_amount_usd'],
        bins=[-1, 1e5, 5e5, 2e6, 1e7, np.inf],
        labels=['<$100K', '$100K-500K', '$500K-2M', '$2M-10M', '$10M+']
    )
    return pd.DataFrame({
        'org_uuid': first_funding['org_uuid'],
        'first_funding_amount_bucket': buckets
    })

funding_bucket_df = bucketize_funding_amounts(
    early_df[['org_uuid', 'announced_on', 'raised_amount_usd', 'funding_round_uuid']].dropna()
)

In [ ]:
"""
Burn Rate: # total EARLY funding / months active
"""
# Ensure dates are datetime
early_df['announced_on'] = pd.to_datetime(early_df['announced_on'], errors='coerce')
early_df['founded_on'] = pd.to_datetime(early_df['founded_on'], errors='coerce')

# Step 1: drop duplicate funding_round_uuids (keep earliest instance)
early_unique = early_df.drop_duplicates(subset='funding_round_uuid').copy()

# Step 2: sum early funding (now deduplicated)
early_unique['raised_amount_usd'] = early_unique['raised_amount_usd'].fillna(0)
early_funding_sum = early_unique.groupby('org_uuid')['raised_amount_usd'].sum().reset_index(name='total_early_funding_usd')

# Step 3: compute last early round date (deduplicated)
last_early_round = early_unique.groupby('org_uuid')['announced_on'].max().reset_index(name='last_early_round_date')

# Step 4: compute months active
burn_timing = early_unique[['org_uuid', 'founded_on']].drop_duplicates().merge(
    last_early_round, on='org_uuid', how='left'
)
burn_timing['active_months'] = (
    (burn_timing['last_early_round_date'] - burn_timing['founded_on']).dt.days / 30
)

# Step 5: calculate burn rate
burn_rate_df = burn_timing.merge(early_funding_sum, on='org_uuid', how='left')
burn_rate_df['burn_rate'] = burn_rate_df['total_early_funding_usd'] / burn_rate_df['active_months']
burn_rate_df = burn_rate_df[['org_uuid', 'burn_rate']]


In [ ]:
"""
Geography
"""
# Normalize hub cities
STARTUP_HUB_CITIES = [c.strip().lower() for c in STARTUP_HUB_CITIES]

# Substring match function
def is_in_startup_hub(city):
    if pd.isna(city):
        return False
    city = str(city).strip().lower()
    return any(hub in city for hub in STARTUP_HUB_CITIES)

# Apply to DataFrame
early_df['is_startup_hub'] = early_df['city'].apply(is_in_startup_hub)

# Extract final startup-level flag
startup_hub_df = early_df[['org_uuid', 'is_startup_hub']].drop_duplicates()

# Is startup in city that is startup rich
# density = # startups founded in same city + year
cohort = early_df[['org_uuid', 'city', 'founded_on']].drop_duplicates()
cohort['founded_year'] = pd.to_datetime(cohort['founded_on'], errors='coerce').dt.year
cohort_density = cohort.groupby(['city', 'founded_year']).size().reset_index(name='cohort_funding_density')
cohort = cohort.merge(cohort_density, on=['city', 'founded_year'], how='left')
cohort_density_df = cohort[['org_uuid', 'cohort_funding_density']]


#### Outcome Variables

In [ ]:
def compute_success_labels(early_df, rounds_df, outcomes_df, SUCCESS_YEARS_1=5, SUCCESS_YEARS_2=10):
    # Clean datetime
    outcomes_df['went_public_on'] = pd.to_datetime(outcomes_df['went_public_on'], errors='coerce')
    outcomes_df['acquired_on'] = pd.to_datetime(outcomes_df['acquired_on'], errors='coerce')
    early_df['founded_on'] = pd.to_datetime(early_df['founded_on'], errors='coerce')
    rounds_df['funding_round_date'] = pd.to_datetime(rounds_df['funding_round_date'], errors='coerce')

    # Base outcome type from status (operating, closed, ipo, acquisition)
    org_df = early_df[['org_uuid', 'founded_on']].copy()
    org_df = org_df.merge(outcomes_df[['org_uuid', 'status']], on='org_uuid', how='left')
    df = org_df.copy()
    df['outcome_type'] = df['status'].str.lower()

    # Define IPO/acquisition outcome_type
    outcomes_df['outcome_type'] = outcomes_df.apply(
        lambda row: 'ipo' if pd.notnull(row['went_public_on'])
        else ('acquisition' if pd.notnull(row['acquired_on']) else None),
        axis=1
    )

    # IPO/acquisition: always success (anytime)
    exit_success_all = outcomes_df[outcomes_df['outcome_type'].isin(['ipo', 'acquisition'])][['org_uuid', 'outcome_type']].drop_duplicates()
    df = df.drop(columns=['outcome_type'], errors='ignore')  # remove preassigned outcome_type to override with correct priority
    df = df.merge(exit_success_all, on='org_uuid', how='left')

    # Series A+ within X years and company still operating
    def series_success_within(years):
        df_funding = rounds_df[rounds_df['investment_type'].str.lower().isin(EARLY_FUNDING_TYPES)].copy()
        df_funding = df_funding.merge(org_df[['org_uuid', 'founded_on', 'status']], on='org_uuid', how='left')
        df_funding['years_since_founding'] = (df_funding['funding_round_date'] - df_funding['founded_on']).dt.days / 365
        df_funding = df_funding[
            (df_funding['years_since_founding'] <= years) &
            (df_funding['status'].str.lower().isin(VALID_ACTIVE_STATUSES))
        ]
        return df_funding[['org_uuid']].drop_duplicates().assign(outcome_type='funding')

    # Compute Series A+ outcomes
    funding_5y = series_success_within(SUCCESS_YEARS_1)
    funding_10y = series_success_within(SUCCESS_YEARS_2)

    # Assign funding-based outcome type if not already success
    df.loc[df['org_uuid'].isin(funding_5y['org_uuid']) & df['outcome_type'].isna(), 'outcome_type'] = 'funded_no_exit'

    # Label too-young companies founded after Jan 1, 2020
    df['too_young'] = df['founded_on'] > pd.to_datetime("2020-01-01")
    df.loc[df['too_young'] & df['outcome_type'].isna(), 'outcome_type'] = 'too_young'

    # Assign 'still_active' if company is operating, older than 5y, and has no success
    five_year_cutoff = pd.to_datetime("2025-01-01") - timedelta(days=SUCCESS_YEARS_1 * 365)
    is_old_enough = df['founded_on'] < five_year_cutoff
    is_operating = df['status'].str.lower().isin(VALID_ACTIVE_STATUSES)
    no_outcome = df['outcome_type'].isna()

    df.loc[no_outcome & is_operating & is_old_enough, 'outcome_type'] = 'still_active'

    # Anything else left must be closed (no success, not too young, not still operating)
    df['outcome_type'] = df['outcome_type'].fillna('closed')

    # Final success labels
    success_5y_ids = pd.concat([exit_success_all, funding_5y])['org_uuid'].unique()
    success_10y_ids = pd.concat([exit_success_all, funding_10y])['org_uuid'].unique()
    df['success_label_5y'] = df['org_uuid'].isin(success_5y_ids)
    df['success_label_10y'] = df['org_uuid'].isin(success_10y_ids)

    return df[['org_uuid', 'success_label_5y', 'success_label_10y', 'outcome_type']]

In [ ]:
success_df = compute_success_labels(early_df, rounds_df, outcomes_df)

#### Merging and Saving

In [ ]:
final = base_orgs.copy()
final = final[final['org_uuid'].isin(valid_orgs)].copy()

# Founders
final = final.merge(founder_feats, on='org_uuid', how='left')
final = final.merge(repeat_founder_df, on='org_uuid', how='left')
final = final.merge(founder_desc_df, on='org_uuid', how='left')

# Define default fills (EXCLUDE founder_gender_diversity and known_gender_count)
founder_fill_defaults = {
    'founder_count': 0,
    'has_top_school_founder': False,
    'num_optimal_degrees': 0,
    'optimal_degree_ratio': 0.0,
    'is_repeat_founder': False
}
final = final.fillna(value=founder_fill_defaults)

# Convert to int after fill
final['founder_count'] = final['founder_count'].astype(int)

# Track missingness
final['founder_gender_missing'] = final['founder_gender_diversity'].isna()
final['founder_degree_missing'] = final['num_optimal_degrees'] == 0
final['founder_school_missing'] = final['has_top_school_founder'] == False
final['founder_desc_missing'] = final['founder_description_blob'].str.strip() == ''

# if we want, drop rows with null values in all founder-related features
# keep only rows with a valid founder description AND at least one structured founder attribute
# final = final[
#     (~final['founder_desc_missing']) &
#     ~(
#         final['founder_gender_missing'] &
#         final['founder_degree_missing'] &
#         final['founder_school_missing']
#     )
# ].copy()

# final = final[
#     (~final['founder_desc_missing']) &                         # must have description
#     (~final['founder_gender_missing']) &                      # must have gender
#     (~final['founder_degree_missing'] | ~final['founder_school_missing'])  # must have degree OR school
# ].copy()

# final = final[
#     ~final['founder_desc_missing']
# ].copy()

# timing
final = final.merge(timing_df[['org_uuid', 'founded_year', 'age_at_first_funding', 'first_funding_delay']], on='org_uuid', how='left')
# final = final.merge(early_counts, on='org_uuid', how='left')
# final = final.merge(burn_rate_df, on='org_uuid', how='left')
# final = final.merge(has_funding_df, on='org_uuid', how='left')
# final = final.merge(known_amounts_df, on='org_uuid', how='left')
# final = final.merge(funding_velocity_df, on='org_uuid', how='left')
# final = final.merge(funding_bucket_df, on='org_uuid', how='left')

# Merge all early funding features at once
early_funding_features_df = (
    early_series_count_df
    .merge(avg_time_to_round_df, on='org_uuid', how='outer')
    .merge(avg_time_between_df, on='org_uuid', how='outer')
    .merge(has_funding_df, on='org_uuid', how='outer')
    .merge(known_amounts_df, on='org_uuid', how='outer')
    .merge(funding_velocity_df, on='org_uuid', how='outer')
    .merge(funding_bucket_df, on='org_uuid', how='outer')
)

# Then merge into final
final = final.merge(early_funding_features_df, on='org_uuid', how='left')
final = final.merge(burn_rate_df, on='org_uuid', how='left')  # burn rate is independent

# geography
final = final.merge(startup_hub_df, on='org_uuid', how='left')
final = final.merge(cohort_density_df, on='org_uuid', how='left')

# investor
final = final.merge(investor_count_df, on='org_uuid', how='left')
final = final.merge(known_investor_df, on='org_uuid', how='left')

# outcome
final = final.merge(success_df, on='org_uuid', how='left')

# clean up final types and missingness
cols_to_drop = [col for col in final.columns if col in ['matched_industries', 'early_rounds_2yr', 'num_early_rounds', 'Unnamed: 0']]
final = final.drop(columns=cols_to_drop, errors='ignore')
final = final.drop_duplicates(subset='org_uuid') # defensive
final = final.reset_index(drop=True)


In [ ]:
cb_final_data_path = os.path.join(OUTPUT_CB_FOLDER, "cb_final_data.csv")
final.to_csv(cb_final_data_path, index=False)

#### Overview

In [ ]:
# overview
final = pd.read_csv(cb_final_data_path)
print(f"final shape: {final.shape}")
print(f"confirming one to one mapping: {final['org_uuid'].is_unique}")
print()
print(final.info())

In [ ]:
# Step 1: Split the industries per startup
final['industry_list'] = final['industry'].str.split(',')

# Step 2: Explode the list into multiple rows
industry_exploded = final.explode('industry_list')

# Step 3: Group by industry and count
industry_counts = industry_exploded.groupby('industry_list').size().reset_index(name='startup_count')

# Step 4: Sort by number of startups (optional, for easier reading)
industry_counts = industry_counts.sort_values('startup_count', ascending=False)

# Display
print(industry_counts)


In [ ]:
print("sample rows")
print(final.head(10))

In [ ]:
res = final[final['org_name'].str.lower() == 'zello']
pprint.pprint(res)

In [ ]:
success_label = 'success_label_5y'
true_count = (final[success_label] == True).sum()
print(true_count)

false_count = (final[success_label] == False).sum()
print(false_count)

# success_label = 'success_label_10y'
# true_count = (final[success_label] == True).sum()
# print(true_count)

# false_count = (final[success_label] == False).sum()
# print(false_count)

majority_guess = (false_count / (true_count + false_count)) * 100
print(f"majority guess: {majority_guess:.2f}%")

In [ ]:
print(final['outcome_type'].unique())
print(final['outcome_type'].value_counts())

In [ ]:
temp = final[final[success_label] == True]
temp.tail(10)

### Economic

#### Feat Eng

In [ ]:
"""
Clean
"""
# ----------- Load and Preprocess Full Dataset -----------
# Full raw dataframe (including macro + ETF rows)
econ_df_full = econ_df.copy()

# Convert to datetime and sort
econ_df_full['date'] = pd.to_datetime(econ_df_full['date'], errors='coerce')
econ_df_full = econ_df_full.sort_values('date')

# Force all adj_close_* columns to numeric
adj_close_cols = [col for col in econ_df_full.columns if col.startswith('adj_close_')]
econ_df_full[adj_close_cols] = econ_df_full[adj_close_cols].apply(pd.to_numeric, errors='coerce')

# ----------- Monthly Macro Dataset -----------
# Drop ETF column and resample all other columns monthly
econ_df_macro = (
    econ_df_full
    .drop(columns=['etf'])  # keep only numeric macro fields
    .set_index('date')
    .resample('MS')
    .mean(numeric_only=True)
    .reset_index()
)

# ----------- Monthly ETF Dataset -----------
# Group by ETF and resample all numeric columns monthly
econ_df_etf = (
    econ_df_full
    .drop(columns=['adj_close_^gspc'])  # avoid duplicate SP500 col
    .set_index('date')
    .groupby('etf')
    .resample('MS')
    .mean(numeric_only=True)
    .reset_index()
)

print(econ_df_macro.dtypes)

# Get unique ETF list
etf_list = econ_df['etf'].dropna().unique()[2:]
print(f"Unique ETFs: {etf_list}")

In [ ]:
def engineer_economic_features(econ_df, date):
    """
    Return macroeconomic features for the given date ([-12m, +3m] window).
    """
    date = pd.Timestamp(date)  # Ensure correct datetime type
    window = econ_df[(econ_df['date'] >= date - pd.DateOffset(months=12)) &
                    (econ_df['date'] <= date + pd.DateOffset(months=3))]

    features = {}
    features['date'] = date

    macro_features = {
        'gdp_growth': ['avg', 'delta'],
        'interest_rate_fed_funds': ['avg', 'delta'],
        'yield_curve_10y_2y': ['avg'],
        'unemployment_rate': ['avg'],
        'cpi_inflation': ['avg'],
        'consumer_sentiment': ['avg', 'z_score'],
        'vix_index': ['max'],
    }

    for col, ops in macro_features.items():
        col_series = window[col].dropna()
        if col_series.empty:
            continue
        if 'avg' in ops:
            features[f'{col}_avg_15m'] = col_series.mean()
        if 'delta' in ops and len(col_series) >= 4:
            features[f'{col}_delta_3m'] = col_series.iloc[-1] - col_series.iloc[-4]
        if 'z_score' in ops and col_series.std() > 0:
            features[f'{col}_z_latest'] = (col_series.iloc[-1] - col_series.mean()) / col_series.std()
        if 'max' in ops:
            features[f'{col}_max_15m'] = col_series.max()

    # binary flags
    features['yield_curve_inversion_flag'] = int((window['yield_curve_10y_2y'] < 0).any())
    features['vix_spike_flag'] = int((window['vix_index'] > 25).any())

    # SP500 as macro momentum indicator using ^GSPC
    gspc = window['adj_close_^gspc'].dropna()
    if len(gspc) >= 3:
        features['sp500_price_change_3m'] = gspc.iloc[-1] - gspc.iloc[-3]
        features['sp500_volatility_3m'] = gspc.pct_change().dropna().std()
        features['sp500_momentum_latest'] = gspc.iloc[-1] - gspc.mean()
    
    # Absolute level indicators
    if 'vix_index' in window.columns:
        features['vix_latest'] = window['vix_index'].dropna().iloc[-1] if not window['vix_index'].dropna().empty else np.nan
    if 'interest_rate_fed_funds' in window.columns:
        features['fed_funds_rate_latest'] = window['interest_rate_fed_funds'].dropna().iloc[-1] if not window['interest_rate_fed_funds'].dropna().empty else np.nan
        
    return features


In [ ]:
def engineer_etf_features(econ_df, date, etf_symbol):
    """
    Return ETF features for a given date and ETF symbol ([-12m, +3m] window).
    """
    date = pd.Timestamp(date)
    df = econ_df[(econ_df['etf'] == etf_symbol) &
                (econ_df['date'] >= date - pd.DateOffset(months=12)) &
                (econ_df['date'] <= date + pd.DateOffset(months=3))]

    if df.empty:
        return {}

    df = df.apply(pd.to_numeric, errors='coerce')  # enforce numeric
    feats = {}
    feats[f'{etf_symbol.lower()}_return_3m'] = df['daily_return'].mean()
    feats[f'{etf_symbol.lower()}_volatility_3m'] = df['volatility'].mean()
    feats[f'{etf_symbol.lower()}_momentum_latest'] = df['momentum'].dropna().iloc[-1] if not df['momentum'].isna().all() else np.nan

    if '50-day_ma' in df.columns and '200-day_ma' in df.columns:
        if not df[['50-day_ma', '200-day_ma']].isna().all().all():
            ma_50 = df['50-day_ma'].iloc[-1]
            ma_200 = df['200-day_ma'].iloc[-1]
            feats[f'{etf_symbol.lower()}_golden_cross_flag'] = int(ma_50 > ma_200)

        price_col = f'adj_close_{etf_symbol.lower()}'
        if price_col in df.columns:
            price_series = df[price_col].dropna()
            if not price_series.empty and not np.isnan(ma_50):
                price = price_series.iloc[-1]
                feats[f'{etf_symbol.lower()}_ma50_to_price_ratio'] = ma_50 / price if price != 0 else np.nan

    return feats

#### Run

In [ ]:
# ----------- Generate Features -----------

# Macro features (1 row per month)
macro_feature_rows = []
for raw_date in tqdm(econ_df_macro['date'].dropna().unique(), desc="Macro"):
    feats = engineer_economic_features(econ_df_macro, raw_date)
    macro_feature_rows.append(feats)
macro_feat_df = pd.DataFrame(macro_feature_rows)

# ETF features (1 row per month, wide format)
etf_feature_rows = []
for raw_date in tqdm(econ_df_etf['date'].dropna().unique(), desc="ETF"):
    date_feats = {'date': raw_date}
    for etf in etf_list:
        etf_feats = engineer_etf_features(econ_df_etf, raw_date, etf)
        date_feats.update(etf_feats)
    etf_feature_rows.append(date_feats)
etf_feat_df = pd.DataFrame(etf_feature_rows)

# Join macro and ETF features on 'date'
econ_features_df = pd.merge(macro_feat_df, etf_feat_df, on='date', how='outer').sort_values('date')


In [ ]:
econ_features_df.info()
pprint.pprint(macro_feat_df.head(n=20))

#### Saving

In [ ]:
# macro_out_path = os.path.join(OUTPUT_ECON_FOLDER, "macro_feature_engineer.csv")
# etf_out_path = os.path.join(OUTPUT_ECON_FOLDER, "etf_feature_engineer.csv")
# macro_feat_df.to_csv(macro_out_path, index=False)
# etf_feat_df.to_csv(etf_out_path, index=False)
# print(f"Saved macro features to: {macro_out_path}")
# print(f"Saved ETF features to: {etf_out_path}")

econ_feat_out_path = os.path.join(OUTPUT_ECON_FOLDER, "economic_feature_engineer.csv")
econ_features_df.to_csv(econ_feat_out_path, index=False)
print(f"Saved macro features to: {econ_feat_out_path}")

#### Quick Test

In [ ]:
def plot_macro_window(econ_features_df, anchor_date, cols_to_plot):
    """
    Plot a 15-month macroeconomic window centered around anchor_date.
    """
    anchor_date = pd.Timestamp(anchor_date)
    window_start = anchor_date - pd.DateOffset(months=12)
    window_end = anchor_date + pd.DateOffset(months=3)

    window = econ_features_df[(econ_features_df['date'] >= window_start) & (econ_features_df['date'] <= window_end)]

    if window.empty:
        print(f"No data found for window around {anchor_date.date()}")
        return

    plt.figure(figsize=(14, len(cols_to_plot) * 3))
    for i, col in enumerate(cols_to_plot, 1):
        plt.subplot(len(cols_to_plot), 1, i)
        plt.plot(window['date'], window[col], marker='o')
        plt.axvline(anchor_date, color='red', linestyle='--', label='Anchor Date')
        plt.title(f"{col} | {window_start.date()} to {window_end.date()}")
        plt.xlabel("Date")
        plt.ylabel(col)
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
cols = ['gdp_growth_avg_15m', 'consumer_sentiment_avg_15m', 'sp500_price_change_3m']
plot_macro_window(econ_features_df, anchor_date='2009-03-01', cols_to_plot=cols)

### Unstructured
- Run on Google Colab for GPUs for LLMs on 
    - Startup Descr + Outlook
    - Founder Descr
- nlp_processing.ipyb for BERT classification

## Feature Selection and Merging

#### Paths

In [ ]:
cb_final_df_path = os.path.join(OUTPUT_CB_FOLDER, "cb_final_data.csv")
econ_final_df_path = os.path.join(OUTPUT_ECON_FOLDER, "economic_feature_engineer.csv")
nlp_sentiment_startup_df_path = os.path.join(OUTPUT_NLP_FOLDER, "nlp_sentiment_features.csv")
nlp_sentiment_outlook_df_path = os.path.join(OUTPUT_NLP_FOLDER, "nlp_outlook_checkpoint.csv")
nlp_outlook_llm_df_path = os.path.join(OUTPUT_NLP_FOLDER, "alignment_scores_clean.csv")
nlp_founder_llm_df_path = os.path.join(OUTPUT_NLP_FOLDER, "founder_strength_scores_clean.csv")
output_merged_df_path = os.path.join(OUTPUT_FINAL_FOLDER, 'merged_startup_data.csv')

In [ ]:
cb_final_df = pd.read_csv(cb_final_df_path)
econ_final_df = pd.read_csv(econ_final_df_path)
nlp_startup_sentiment_df = pd.read_csv(nlp_sentiment_startup_df_path)
nlp_outlook_sentiment_df = pd.read_csv(nlp_sentiment_outlook_df_path)
nlp_outlook_llm_df = pd.read_csv(nlp_outlook_llm_df_path)
nlp_founder_llm_df = pd.read_csv(nlp_founder_llm_df_path)

### Load

In [ ]:
print("Crunchbase Final Data:\n")
print(cb_final_df.info())
print(cb_final_df.head(2))
print()
print("Economic Final Data:\n")
print(econ_final_df.info())
print(econ_final_df.head(2))
print()
print("NLP Startup Sentiment Data:\n")
print(nlp_startup_sentiment_df.info())
print(nlp_startup_sentiment_df.head(2))
print()
print("NLP Outlook Sentiment Data:\n")
print(nlp_outlook_sentiment_df.info())
print(nlp_outlook_sentiment_df.head(2))
print()
print("NLP Outlook LLM Data:\n")
print(nlp_outlook_llm_df.info())
print(nlp_outlook_llm_df.head(2))
print()
print("NLP Founder LLM Data:\n")
print(nlp_founder_llm_df.info())
print(nlp_founder_llm_df.head(2))

### Constants

In [ ]:
CB_FEATURES_TO_KEEP = [
    # Identifiers & Merge Keys
    'org_uuid', 'founded_on', 'founded_year', 'industry',
    # Target Variables
    'success_label_5y', 'success_label_10y',
    # Analysis Variables
    'org_name', 'outcome_type', 'city', # Keep city for context/viz
    # Features
    'founder_count', 'founder_gender_diversity', 'has_top_school_founder',
    'num_optimal_degrees', 'optimal_degree_ratio', 'is_repeat_founder',
    'founder_gender_missing', 'founder_degree_missing', 'founder_school_missing',
    'founder_desc_missing', 'age_at_first_funding', 'first_funding_delay',
    'early_series_count', 'avg_time_to_early_round_months', 'avg_time_between_rounds',
    'burn_rate', 'has_funding_data','num_disclosed_rounds', 'has_disclosed_funding', 
    'funding_velocity','first_funding_amount_bucket', 'is_startup_hub', 
    'cohort_funding_density', 'investor_count', 'has_known_investor'
    # Raw text descriptions and some derived columns are dropped as planned
]

BUCKET_TO_ETF = { # Lowercase keys
    "technology": "xlk", "life sciences": "vht", "cleantech": "pbw",
    "consumer goods": "xly", "transportation": "iyt", "media entertainment and gaming": "vox",
    "telecom": "iyz", "real estate": "vnq", "fintech": "finx"
}
ETF_METRIC_SUFFIXES = [
    '_return_3m', '_volatility_3m', '_momentum_latest',
    '_golden_cross_flag', '_ma50_to_price_ratio'
]

# *** DEFINE COLUMNS TO KEEP FROM PRE-CALCULATED ECONOMIC DATA ***
GENERAL_ECON_FEATURES_TO_KEEP = [
    'gdp_growth_avg_15m', 'gdp_growth_delta_3m',
    'interest_rate_fed_funds_avg_15m', 'interest_rate_fed_funds_delta_3m', 'fed_funds_rate_latest',
    'yield_curve_10y_2y_avg_15m', 'yield_curve_inversion_flag',
    'unemployment_rate_avg_15m',
    'cpi_inflation_avg_15m',
    'consumer_sentiment_avg_15m', 'consumer_sentiment_z_latest',
    'vix_index_max_15m', 'vix_spike_flag', 'vix_latest',
    'sp500_price_change_3m', 'sp500_volatility_3m', 'sp500_momentum_latest',
]
INDIVIDUAL_ETF_FEATURES_TO_KEEP = []
etf_tickers = list(set(BUCKET_TO_ETF.values())) # Get unique lowercase tickers
for ticker in etf_tickers:
    for suffix in ETF_METRIC_SUFFIXES:
        INDIVIDUAL_ETF_FEATURES_TO_KEEP.append(f"{ticker}{suffix}")

ALL_ECON_COLS_TO_KEEP = ['date'] + GENERAL_ECON_FEATURES_TO_KEEP + INDIVIDUAL_ETF_FEATURES_TO_KEEP


### Merging

#### Data Preperation

In [ ]:
# --- 2. Data Preparation Functions ---

def prepare_crunchbase(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the Crunchbase dataframe:
    - Selects columns based on CB_FEATURES_TO_KEEP.
    - Converts 'founded_on' to datetime.
    - Imputes missing/invalid 'founded_on' using start of 'founded_year'.
    - Creates 'merge_date' column for joining with economic data.
    - Drops rows if a merge date cannot be determined.
    - Sorts by 'merge_date'.

    Args:
        df: The raw Crunchbase dataframe.

    Returns:
        A prepared and sorted dataframe ready for merging.
    """
    print("Preparing Crunchbase data...")
    # Assume CB_FEATURES_TO_KEEP is defined globally/passed as argument
    cols_to_keep = CB_FEATURES_TO_KEEP
    cols_to_keep = [col for col in cols_to_keep if col in df.columns]
    prepared_df = df[cols_to_keep].copy()
    initial_rows = len(prepared_df)
    print(f"Initial CB rows: {initial_rows}")

    # Attempt to convert founded_on to datetime
    prepared_df['founded_on_dt'] = pd.to_datetime(prepared_df['founded_on'], errors='coerce')

    # Ensure founded_year is numeric (needed for imputation)
    prepared_df['founded_year_num'] = pd.to_numeric(prepared_df['founded_year'], errors='coerce').astype('Int64')

    # Identify rows where founded_on_dt is NaT (missing or invalid)
    missing_date_mask = prepared_df['founded_on_dt'].isna()
    num_missing_dates = missing_date_mask.sum()

    if num_missing_dates > 0:
        print(f"Found {num_missing_dates} rows with missing/invalid 'founded_on'. Imputing using start of 'founded_year'.")
        # Only impute where year is valid
        can_impute_mask = missing_date_mask & prepared_df['founded_year_num'].notna()
        num_can_impute = can_impute_mask.sum()
        # Create the imputed date string 'YYYY-01-01'
        imputed_dates = pd.to_datetime(prepared_df.loc[can_impute_mask, 'founded_year_num'].astype(str) + '-01-01',
                                    format='%Y-%m-%d', errors='coerce')
        # Fill NaT values in founded_on_dt with the imputed dates
        prepared_df.loc[can_impute_mask, 'founded_on_dt'] = imputed_dates
        num_failed_impute = num_missing_dates - num_can_impute
        if num_failed_impute > 0:
            print(f"Warning: Could not impute date for {num_failed_impute} rows due to missing/invalid 'founded_year'.")

    # Drop rows where founded_on_dt is STILL NaT after imputation (date is unrecoverable)
    final_rows_before_drop = len(prepared_df)
    prepared_df = prepared_df.dropna(subset=['founded_on_dt'])
    final_rows_after_drop = len(prepared_df)
    if final_rows_after_drop < final_rows_before_drop:
        rows_dropped = final_rows_before_drop - final_rows_after_drop
        print(f"Dropped {rows_dropped} rows where date could not be determined (missing/invalid founded_on AND founded_year).")

    # CRITICAL: Sort by the final, imputed founded_on_dt for merge_asof
    prepared_df = prepared_df.sort_values(by='founded_on_dt').reset_index(drop=True)

    # Clean up helper columns and rename date column for clarity
    prepared_df = prepared_df.drop(columns=['founded_year_num'])
    prepared_df = prepared_df.rename(columns={'founded_on_dt': 'merge_date'})

    print(f"Crunchbase data prepared, imputed, and sorted. Final Shape: {prepared_df.shape}")
    return prepared_df

def prepare_economic_data_precalculated(df: pd.DataFrame, columns_to_keep: List[str]) -> pd.DataFrame:
    """
    Prepares pre-calculated economic data: selects specific columns,
    converts date, sorts by date.

    Args:
        df: The raw economic dataframe with pre-calculated features.
        columns_to_keep: List of column names to select (must include 'date').

    Returns:
        A prepared dataframe with selected columns, sorted by date.
    """
    print("Preparing pre-calculated Economic data...")

    # Ensure all requested columns exist in the input dataframe
    actual_cols_to_keep = [col for col in columns_to_keep if col in df.columns]
    missing_cols = set(columns_to_keep) - set(actual_cols_to_keep)
    if missing_cols:
        print(f"Warning: The following expected economic columns were not found and will be skipped: {missing_cols}")
    if 'date' not in actual_cols_to_keep :
        raise ValueError("Critical Error: 'date' column is missing from the economic data frame.")

    econ_prepared = df[actual_cols_to_keep].copy() # Select only specified columns

    econ_prepared['date'] = pd.to_datetime(econ_prepared['date'], errors='coerce')
    econ_prepared = econ_prepared.dropna(subset=['date']) # Drop rows if date conversion failed
    econ_prepared = econ_prepared.sort_values(by='date').reset_index(drop=True) # Sort by date
    # Handle potential duplicate dates if necessary
    if econ_prepared.duplicated(subset=['date']).any():
        print("Warning: Duplicate dates found in economic data. Keeping first occurrence.")
        econ_prepared = econ_prepared.drop_duplicates(subset=['date'], keep='first')

    print(f"Pre-calculated Economic data prepared, selected, and sorted. Shape: {econ_prepared.shape}")
    return econ_prepared

def engineer_averaged_etf_features(row: pd.Series, bucket_to_etf_map: Dict[str, str], metric_suffixes: List[str]) -> pd.Series:
    """
    Calculates average ETF metrics for a startup based on its industry
    using the pre-calculated ETF columns already merged onto the row.

    Args:
        row: A row from the merged dataframe (after merge_asof).
        bucket_to_etf_map: Dictionary mapping industry buckets (lowercase) to ETF tickers (lowercase).
        metric_suffixes: List of suffixes for ETF metrics (e.g., '_return_3m').

    Returns:
        A Pandas Series containing the averaged ETF metrics for the startup.
    """
    industry_str = row.get('industry')
    averaged_features = {}

    # Initialize features with NaN
    for suffix in metric_suffixes: averaged_features[f'avg_etf{suffix}'] = np.nan

    if pd.isna(industry_str) or not isinstance(industry_str, str):
        return pd.Series(averaged_features, dtype='float64')

    # Parse industries (lowercase) and find corresponding ETFs (lowercase)
    industries = [ind.strip().lower() for ind in industry_str.split(',')]
    relevant_etfs = list(set(bucket_to_etf_map.get(ind) for ind in industries if ind in bucket_to_etf_map)) # Use .get for safety

    if not relevant_etfs:
        return pd.Series(averaged_features, dtype='float64')

    # Calculate average for each metric type across relevant ETFs
    for suffix in metric_suffixes:
        metric_values = []
        for etf_ticker in relevant_etfs:
            col_name = f"{etf_ticker}{suffix}" # Construct column name (e.g., xlk_return_3m)
            # Check if column exists in the row and if the value is not NaN
            if col_name in row and pd.notna(row[col_name]):
                metric_values.append(row[col_name])
        # Calculate average if we collected any values
        if metric_values:
            averaged_features[f'avg_etf{suffix}'] = np.mean(metric_values)
        # else: feature remains NaN (already initialized)

    return pd.Series(averaged_features, dtype='float64')

def prepare_nlp_startup_sentiment(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the NLP startup/founder sentiment data by selecting
    FinBERT signed scores and similarity scores, then renaming columns.

    Args:
        df: The raw NLP startup/founder sentiment dataframe.

    Returns:
        A prepared dataframe with selected and renamed columns.
    """
    print("Preparing NLP Startup/Founder Sentiment data...")
    cols_to_keep = [
        'org_uuid',
        'org_finbert_numeric_signed',
        'founder_sentiment_numeric_signed',
        'org_sim_to_exemplar',
        'founder_sim_to_exemplar'
    ]
    # Ensure all selected columns exist
    cols_to_keep = [col for col in cols_to_keep if col in df.columns]
    prepared_df = df[cols_to_keep].copy()

    rename_dict = {
        'org_finbert_numeric_signed': 'org_desc_sentiment_finbert',
        'founder_sentiment_numeric_signed': 'founder_desc_sentiment_finbert',
        'org_sim_to_exemplar': 'org_desc_sim_exemplar',
        'founder_sim_to_exemplar': 'founder_desc_sim_exemplar'
    }
    prepared_df = prepared_df.rename(columns=rename_dict)
    print(f"NLP Startup/Founder Sentiment data prepared. Shape: {prepared_df.shape}")
    return prepared_df

def prepare_llm_founder_score(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the LLM founder score data. Assumes pre-processing handles
    duplicates, otherwise add .drop_duplicates(subset=['org_uuid'], keep='first') here.

    Args:
        df: The raw LLM founder score dataframe.

    Returns:
        A prepared dataframe with selected and renamed columns.
    """
    print("Preparing LLM Founder Score data...")
    # Note: Assuming the input df is already pre-processed to have one score per org_uuid.
    # If not, uncomment the following line:
    # df = df.drop_duplicates(subset=['org_uuid'], keep='first')

    cols_to_keep = ['org_uuid', 'score']
    # Ensure all selected columns exist
    cols_to_keep = [col for col in cols_to_keep if col in df.columns]
    prepared_df = df[cols_to_keep].copy()

    prepared_df = prepared_df.rename(columns={'score': 'llm_founder_score'})

    # Handle potential NaN scores if needed (e.g., fill with median or specific value)
    # Example: prepared_df['llm_founder_score'] = prepared_df['llm_founder_score'].fillna(prepared_df['llm_founder_score'].median())

    print(f"LLM Founder Score data prepared. Shape: {prepared_df.shape}")
    return prepared_df

def prepare_llm_outlook_alignment(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the LLM outlook alignment data by aggregating scores per org_uuid,
    calculating the mean, and creating a binned version.

    Args:
        df: The raw LLM outlook alignment dataframe.

    Returns:
        A prepared dataframe with aggregated and binned scores.
    """
    print("Preparing LLM Outlook Alignment data...")
    # Ensure score is numeric, coerce errors to NaN
    df['score'] = pd.to_numeric(df['score'], errors='coerce')
    # Drop rows where score could not be parsed or is missing before aggregation
    df = df.dropna(subset=['score'])

    # Aggregate: Calculate mean score per org_uuid
    agg_df = df.groupby('org_uuid')['score'].mean().reset_index()
    agg_df = agg_df.rename(columns={'score': 'llm_outlook_align_score_avg'})

        # Binning: Create binned score (-1, 0, 1)
    # Adjusted conditions to cover averages between 4 and 5
    conditions = [
        agg_df['llm_outlook_align_score_avg'] < 3.5,         # Scores 1, 2, 3 and averages < 4
        (agg_df['llm_outlook_align_score_avg'] >= 3.5) & (agg_df['llm_outlook_align_score_avg'] < 4.5),         # Score 4 and averages >= 4 but < 5
        agg_df['llm_outlook_align_score_avg'] >= 4.5         # Score 5 and averages == 5
    ]
    choices = [-1, 0, 1]
    agg_df['llm_outlook_align_score_binned'] = np.select(conditions, choices, default=np.nan) # Default NaN handles any unexpected cases or NaNs in avg

    print(f"LLM Outlook Alignment data prepared. Shape: {agg_df.shape}")
    return agg_df[['org_uuid', 'llm_outlook_align_score_avg', 'llm_outlook_align_score_binned']]

def prepare_nlp_outlook_sentiment(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the NLP outlook sentiment data by calculating FinBERT net scores
    for outlook and timing signal sections, selecting relevant columns, and renaming.

    Args:
        df: The raw NLP outlook sentiment dataframe.

    Returns:
        A prepared dataframe with industry/year FinBERT net sentiment scores.
    """
    print("Preparing NLP Outlook Sentiment data...")

    # Calculate FinBERT net sentiment scores
    # Ensure necessary columns exist before calculation
    if 'outlook_finbert_positive' in df.columns and 'outlook_finbert_negative' in df.columns:
        df['industry_outlook_sentiment_finbert'] = df['outlook_finbert_positive'] - df['outlook_finbert_negative']
    else:
        print("Warning: Outlook FinBERT columns not found. Setting sentiment to NaN.")
        df['industry_outlook_sentiment_finbert'] = np.nan

    if 'timing_signal_finbert_positive' in df.columns and 'timing_signal_finbert_negative' in df.columns:
        df['industry_timing_sentiment_finbert'] = df['timing_signal_finbert_positive'] - df['timing_signal_finbert_negative']
    else:
        print("Warning: Timing Signal FinBERT columns not found. Setting sentiment to NaN.")
        df['industry_timing_sentiment_finbert'] = np.nan

    cols_to_keep = ['industry', 'year', 'industry_outlook_sentiment_finbert', 'industry_timing_sentiment_finbert']
    # Filter out columns that weren't created if source columns were missing
    cols_to_keep = [col for col in cols_to_keep if col in df.columns]

    prepared_df = df[cols_to_keep].copy()

    # Normalize industry names for merging (lowercase, strip whitespace)
    if 'industry' in prepared_df.columns:
        prepared_df['industry'] = prepared_df['industry'].str.lower().str.strip()
    else:
        print("Warning: 'industry' column not found in NLP Outlook Sentiment data.")
        # Handle appropriately - maybe return empty df or raise error depending on strictness needed
        return pd.DataFrame(columns=cols_to_keep) # Return empty structure

    print(f"NLP Outlook Sentiment data prepared. Shape: {prepared_df.shape}")
    return prepared_df


In [ ]:
# --- 3. Helper Functions for Merging ---

def get_primary_industry(industry_str: str) -> Optional[str]:
    """
    Extracts the first industry from a comma-separated string.

    Args:
        industry_str: Comma-separated string of industries.

    Returns:
        The first industry found, or None if input is invalid.
    """
    if pd.isna(industry_str) or not isinstance(industry_str, str) or not industry_str.strip():
        return None
    return industry_str.split(',')[0].strip()

#### Merging Logic

In [ ]:
# --- 4. Merging Logic ---

def merge_datasets(cb_df: pd.DataFrame, # Sorted by merge_date
                    econ_df: pd.DataFrame, # Sorted by date, pre-calculated & selected features
                    nlp_startup_df: pd.DataFrame,
                    llm_founder_df: pd.DataFrame,
                    llm_outlook_df: pd.DataFrame,
                    nlp_outlook_df: pd.DataFrame,
                    bucket_to_etf_map: Dict[str, str],
                    etf_metric_suffixes: List[str]
                    ) -> pd.DataFrame:
    """
    Merges prepared dataframes into a single master dataframe.
    - Uses pd.merge_asof to join selected pre-calculated economic data.
    - Merges NLP/LLM datasets based on org_uuid.
    - Merges NLP Outlook data based on founded_year and primary industry.
    - Engineers averaged ETF features from merged individual ETF columns.
    - Cleans up by dropping individual ETF columns.

    Args:
        cb_df: Prepared Crunchbase data (sorted by 'merge_date').
        econ_df: Prepared Economic data with selected features (sorted by 'date').
        nlp_startup_df: Prepared NLP Startup/Founder Sentiment data.
        llm_founder_df: Prepared LLM Founder Score data.
        llm_outlook_df: Prepared LLM Outlook Alignment data.
        nlp_outlook_df: Prepared NLP Outlook Sentiment data.
        bucket_to_etf_map: Dictionary mapping industry buckets (lowercase) to ETF tickers (lowercase).
        etf_metric_suffixes: List of suffixes for ETF metrics (e.g., '_return_3m').

    Returns:
        The final merged and cleaned master dataframe.
    """
    print("Starting dataset merging...")

    # 1. Merge Economic Data using merge_asof
    print("Performing merge_asof for economic data...")
    master_df = pd.merge_asof(
        cb_df,
        econ_df,
        left_on='merge_date', # Use the potentially imputed date from CB
        right_on='date',      # Use the date from Econ data
        direction='backward'  # Use data on or before the merge_date
    )
    if 'date' in master_df.columns: master_df = master_df.drop(columns=['date']) # Drop redundant date column
    print(f"Shape after merging Economic Data: {master_df.shape}")
    if econ_df.shape[1] > 1 : # Check if econ_df had columns other than 'date'
        first_econ_col = econ_df.columns[1];
        if first_econ_col in master_df.columns:
            failed_merge_count = master_df[first_econ_col].isnull().sum()
            if failed_merge_count > 0: print(f"Info: {failed_merge_count} rows have NaN economic features (check merge_asof results / original data).")

    # 2. Merge NLP/LLM data on org_uuid
    master_df = pd.merge(master_df, nlp_startup_df, on='org_uuid', how='left')
    master_df = pd.merge(master_df, llm_founder_df, on='org_uuid', how='left')
    master_df = pd.merge(master_df, llm_outlook_df, on='org_uuid', how='left')
    print(f"Shape after merging NLP/LLM data: {master_df.shape}")

    # 3. Merge NLP Outlook Sentiment (on founded_year and primary_industry)
    master_df['primary_industry'] = master_df['industry'].apply(get_primary_industry)
    master_df['primary_industry_normalized'] = master_df['primary_industry'].str.lower().str.strip()
    master_df = pd.merge(master_df, nlp_outlook_df,
                        left_on=['founded_year', 'primary_industry_normalized'],
                        right_on=['year', 'industry'],
                        how='left',
                        suffixes=('', '_nlp_outlook'))
    master_df = master_df.drop(columns=['year', 'industry_nlp_outlook', 'primary_industry', 'primary_industry_normalized'], errors='ignore')
    print(f"Shape after merging NLP Outlook Sentiment: {master_df.shape}")

    # 4. Engineer Averaged ETF Features
    print("Engineering averaged ETF features...")
    averaged_etf_features = master_df.apply(
        engineer_averaged_etf_features,
        axis=1,
        args=(bucket_to_etf_map, etf_metric_suffixes) # Pass map and suffixes
    )
    master_df = pd.concat([master_df, averaged_etf_features], axis=1)
    print(f"Shape after adding averaged ETF features: {master_df.shape}")

    # 5. Cleanup: Drop individual ETF columns
    print("Dropping individual ETF columns...")
    cols_to_drop = []
    etf_tickers = list(set(bucket_to_etf_map.values())) # Get unique lowercase tickers
    for ticker in etf_tickers:
        for suffix in etf_metric_suffixes:
            col_name = f"{ticker}{suffix}"
            if col_name in master_df.columns:
                cols_to_drop.append(col_name)
    if cols_to_drop:
        master_df = master_df.drop(columns=cols_to_drop)
        print(f"Dropped {len(cols_to_drop)} individual ETF columns.")
    else: print("No individual ETF columns found to drop.")

    print("Merging and ETF feature engineering completed.")
    return master_df

#### Run and Save

In [ ]:
# Prepare Individual DataFrames
cb_prepared = prepare_crunchbase(cb_final_df)
nlp_startup_prepared = prepare_nlp_startup_sentiment(nlp_startup_sentiment_df)
llm_founder_prepared = prepare_llm_founder_score(nlp_founder_llm_df)
llm_outlook_prepared = prepare_llm_outlook_alignment(nlp_outlook_llm_df)
econ_prepared = prepare_economic_data_precalculated(econ_final_df, ALL_ECON_COLS_TO_KEEP)
nlp_outlook_prepared = prepare_nlp_outlook_sentiment(nlp_outlook_sentiment_df)

# Merge DataFrames using the updated merge_datasets function
master_df = merge_datasets(
    cb_prepared,
    econ_prepared, # Pass the prepared economic data
    nlp_startup_prepared,
    llm_founder_prepared,
    llm_outlook_prepared,
    nlp_outlook_prepared,
    BUCKET_TO_ETF, # Pass the mapping
    ETF_METRIC_SUFFIXES # Pass the suffixes
)

# Final Checks and Save
print("\n--- Final Merged DataFrame Info ---")
final_expected_rows = len(cb_prepared) # Expected rows after CB prep
print(f"Final Shape: {master_df.shape}")
print(f"Expected rows based on prepared CB data: {final_expected_rows}")
# Check if the number of rows matches the prepared CB data
if master_df.shape[0] != final_expected_rows:
    print(f"Warning: Row count mismatch! Expected {final_expected_rows}, got {master_df.shape[0]}. Check merge steps.")
else:
    print(f"Row count matches expected ({master_df.shape[0]} rows).")

# Display info and missing values
print(master_df.info(verbose=True, max_cols=200)) # Show more columns if needed
print("\nMissing Values per Column (Showing columns with any NaNs):")
null_counts = master_df.isnull().sum(); print(null_counts[null_counts > 0].sort_values(ascending=False))

print("\nSample of Final Data (First 5 Rows):")
print(master_df.head())

# Save the final dataframe
master_df.to_csv(output_merged_df_path, index=False)
print(f"\nSuccessfully saved the merged dataset to: {output_merged_df_path}")

In [ ]:
temp = final[final['org_name'] ==  'Meta']
print(temp['age_at_first_funding'])
print(temp['first_funding_delay'])
print(temp['early_series_count'])
print(temp['avg_time_to_early_round_months'])
print(temp['avg_time_between_rounds'])
print(temp['industry'])

In [ ]:
final_dataset = pd.read_csv(output_merged_df_path)
print(final_dataset.info())